## Additional

아래에서는 추가적인 분석을 시도하였다. 내용은 아래와 같다. 

+ **Simpson's Paradox** : 전체 데이터를 고려했을 때와 개별 데이터를 고려했을 때의 관계 역전 현상

+ Hazard : 전체/피험자별 hazard를 도출하고 왜 **200ms** 부근의 특이적 peak가 관찰되는 현상에 대한 설명

+ empirical CDF vs. Survival Plot

+ Cox PH vs. AFT comparison

### Simpson's Paradox

### Hazard

그림이 이상한 피험자가 왜 이리 많지?

![](../EyeTracking_data/Figures/Hazards.png)

+ 그룹별 Hazard의 비교 : 독특한 피험자들 위주로

### emp CDF vs. Survival Plot